# Web Scraping - Dados Segurança Pública de SP

### Resumo

> **Objetivos:** retirar de maneira automática os dados de segurança pública de SP
>
> **Atualização:** 

In [37]:
#-- importando libs
import pandas as pd
import numpy as np
import requests
import time
import csv
import os
import math

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [22]:
#-- carregando o site pelo beautifulsoup
html = urlopen('http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')
bsObj = BeautifulSoup(html)

In [23]:
#-- printando a lista de delegacias
teste = bsObj.select('#conteudo_ddlDelegacias')
for name in teste:
    print(name.get_text())


Todos
001 DP - Sé
001 DP Ferraz de Vasconcelos
002 DP - Bom Retiro
003 DP - Campos Elísios
004 DP - Consolação
005 DP - Aclimação
006 DP - Cambuci
007 DP - Lapa
008 DP - Brás
009 DP - Carandiru
01 DP - Americana
01 DP - Andradina
01 DP - Araraquara
01 DP - Atibaia
01 DP - Avaré
01 DP - Barra do Turvo
01 DP - Barretos
01 DP - Bauru
01 DP - Bebedouro
01 DP - Birigui
01 DP - Botucatu
01 DP - Cabreuva
01 DP - Campinas
01 DP - Campos do Jordão
01 DP - Capão Bonito
01 DP - Cruzeiro
01 DP - Cubatão
01 DP - Espírito Sto. do Pinhal
01 DP - Garça
01 DP - Guaíra
01 DP - Guaratinguetá
01 DP - Guarujá
01 DP - Indaiatuba
01 DP - Itapetininga
01 DP - Itapeva
01 DP - Itapira
01 DP - Itararé
01 DP - Itatiba
01 DP - Itú
01 DP - Ituverava
01 DP - Jaboticabal
01 DP - Jacareí
01 DP - Jaú
01 DP - Leme
01 DP - Lins
01 DP - Lorena
01 DP - Matao
01 DP - Mirassol
01 DP - Mogi Mirim
01 DP - Mogi-Guaçu
01 DP - Olímpia
01 DP - Osvaldo Cruz
01 DP - Penapolis
01 DP - Peruíbe
01 DP - Pindamonhangaba
01 DP - Pirajú
0

In [24]:
browser = webdriver.Chrome()
browser.get('http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')
time.sleep(2)

element = Select(browser.find_element_by_xpath("//select[@name='ctl00$conteudo$ddlAnos']"))
element.select_by_visible_text('2017')

time.sleep(2)

element = Select(browser.find_element_by_xpath("//select[@name='ctl00$conteudo$ddlRegioes']"))
element.select_by_visible_text('Capital')

time.sleep(2)

writer = csv.writer(open("file.csv", 'w'))
dropdown_elements = browser.find_elements_by_xpath("//select[@name='ctl00$conteudo$ddlDelegacias']/option")

for element in dropdown_elements:
    print(element.text)

for element in dropdown_elements:
    writer.writerow(element.text)

Todos
001 DP - Sé
002 DP - Bom Retiro
003 DP - Campos Elísios
004 DP - Consolação
005 DP - Aclimação
006 DP - Cambuci
007 DP - Lapa
008 DP - Brás
009 DP - Carandiru
01 DP Pessoa com Deficiência
010 DP - Penha de França
011 DP - Santo Amaro
012 DP - Pari
013 DP - Casa Verde
014 DP - Pinheiros
015 DP - Itaim Bibi
016 DP - Vila Clementino
017 DP - Ipiranga
018 DP - Alto da Moóca
019 DP - Vila Maria
020 DP - Água Fria
021 DP - Vila Matilde
022 DP - São Miguel Paulista
023 DP - Perdizes
024 DP - Ponte Rasa
025 DP - Parelheiros
026 DP - Sacomã
027 DP - Campo Belo
028 DP - Freguesia do Ó
029 DP - Vila Diva
030 DP - Tatuapé
031 DP - Vila Carrão
032 DP - Itaquera
033 DP - Pirituba
034 DP - Vila Sonia
035 DP - Jabaquara
036 DP - Vila Mariana
037 DP - Campo Limpo
038 DP - Vila Amália
039 DP - Vila Gustavo
040 DP - Vila Santa Maria
041 DP - Vila Rica
042 DP - Parque São Lucas
043 DP - Cidade Ademar
044 DP - Guaianazes
045 DP - Vila Brasilândia
046 DP - Perus
047 DP - Capão Redondo
048 DP - Cidade 

In [24]:
#-- carregando a lista de dps que devem ter seus dados carregados
db_dps = pd.read_excel('Lista DPs.xlsx', sheet_name='Planilha1')
db_dps.head()

,DPs,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id,Lat,Long,LatW,LongW
0,001 DP - Sé,1? DISTRITO POLICIAL DE S?O PAULO,R DA GLORIA 410,LIBERDADE,NaN,7,NaN,4,NaN,NaN,150240,333245.3157,7393899.938,-23.5571,-46.6342
1,002 DP - Bom Retiro,2? DISTRITO POLICIAL - BOM RETIRO,R JARAGUA 383,BOM RETIRO,3317001.0,7,NaN,4,NaN,NaN,150171,332177.5492,7397656.287,-23.5231,-46.6443
2,003 DP - Campos Elísios,3? DISTRITO POLICIAL - CAMPOS ELISEOS,R AURORA 322,SANTA EFIGENIA,1209000.0,7,NaN,4,NaN,NaN,150177,332643.5483,7395832.642,-23.5396,-46.6399
3,004 DP - Consolação,4? DISTRITO POLICIAL DA CONSOLACAO,R MARQUES DE PARANAGUA 246,CONSOLACAO,1303050.0,7,NaN,4,NaN,NaN,150227,331639.9265,7394586.771,-23.5508,-46.6499
4,005 DP - Aclimação,5? DISTRITO POLICIAL - ACLIMACAO,R PROF ANTONIO PRUDENTE 160,LIBERDADE,1509010.0,7,NaN,4,NaN,NaN,150166,332850.2806,7392968.144,-23.5655,-46.6382


In [49]:
#-- printando as dimensões do db
db_dps.shape

(93, 15)

In [26]:
for i in range(0, db_dps.shape[0]):
    browser = webdriver.Chrome()
    browser.get('http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')
    time.sleep(2)

    element = Select(browser.find_element_by_xpath("//select[@name='ctl00$conteudo$ddlAnos']"))
    element.select_by_visible_text('2019')

    time.sleep(2)

    element = Select(browser.find_element_by_xpath("//select[@name='ctl00$conteudo$ddlDelegacias']"))
    element.select_by_visible_text(str(db_dps.iloc[i, 0]))

    time.sleep(2)

    browser.find_element_by_link_text("Ocorrências Registradas por Mês").click()

    #try:
    #    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'conteudo_repAnos_gridDados_0')))
    #except TimeoutException:
    #    pass  # Handle the exception here
    #table = browser.find_element_by_id('conteudo_repAnos_gridDados_0').get_attribute('innerHTML')
    df = pd.read_html(browser.find_element_by_id('conteudo_repAnos_gridDados_0').get_attribute('outerHTML'))

    time.sleep(2)

    browser.quit()
    
    df1 = pd.DataFrame(df[0])
    df1.to_excel('dp'+str(db_dps.iloc[i, 0])+'.xlsx')

In [3]:
#-- listando a pasta com os arquivos do tutela
arr = os.listdir('C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/')
arr

['.ipynb_checkpoints',
 'chromedriver.exe',
 'dp001 DP - Sé.xlsx',
 'dp002 DP - Bom Retiro.xlsx',
 'dp003 DP - Campos Elísios.xlsx',
 'dp004 DP - Consolação.xlsx',
 'dp005 DP - Aclimação.xlsx',
 'dp006 DP - Cambuci.xlsx',
 'dp007 DP - Lapa.xlsx',
 'dp008 DP - Brás.xlsx',
 'dp009 DP - Carandiru.xlsx',
 'dp010 DP - Penha de França.xlsx',
 'dp011 DP - Santo Amaro.xlsx',
 'dp012 DP - Pari.xlsx',
 'dp013 DP - Casa Verde.xlsx',
 'dp014 DP - Pinheiros.xlsx',
 'dp015 DP - Itaim Bibi.xlsx',
 'dp016 DP - Vila Clementino.xlsx',
 'dp017 DP - Ipiranga.xlsx',
 'dp018 DP - Alto da Moóca.xlsx',
 'dp019 DP - Vila Maria.xlsx',
 'dp020 DP - Água Fria.xlsx',
 'dp021 DP - Vila Matilde.xlsx',
 'dp022 DP - São Miguel Paulista.xlsx',
 'dp023 DP - Perdizes.xlsx',
 'dp024 DP - Ponte Rasa.xlsx',
 'dp025 DP - Parelheiros.xlsx',
 'dp026 DP - Sacomã.xlsx',
 'dp027 DP - Campo Belo.xlsx',
 'dp028 DP - Freguesia do Ó.xlsx',
 'dp029 DP - Vila Diva.xlsx',
 'dp030 DP - Tatuapé.xlsx',
 'dp031 DP - Vila Carrão.xlsx',
 'dp0

In [6]:
#-- parsing just xls files
files_xlsx = [f for f in arr if f[:2] == 'dp']
files_xlsx

['dp001 DP - Sé.xlsx',
 'dp002 DP - Bom Retiro.xlsx',
 'dp003 DP - Campos Elísios.xlsx',
 'dp004 DP - Consolação.xlsx',
 'dp005 DP - Aclimação.xlsx',
 'dp006 DP - Cambuci.xlsx',
 'dp007 DP - Lapa.xlsx',
 'dp008 DP - Brás.xlsx',
 'dp009 DP - Carandiru.xlsx',
 'dp010 DP - Penha de França.xlsx',
 'dp011 DP - Santo Amaro.xlsx',
 'dp012 DP - Pari.xlsx',
 'dp013 DP - Casa Verde.xlsx',
 'dp014 DP - Pinheiros.xlsx',
 'dp015 DP - Itaim Bibi.xlsx',
 'dp016 DP - Vila Clementino.xlsx',
 'dp017 DP - Ipiranga.xlsx',
 'dp018 DP - Alto da Moóca.xlsx',
 'dp019 DP - Vila Maria.xlsx',
 'dp020 DP - Água Fria.xlsx',
 'dp021 DP - Vila Matilde.xlsx',
 'dp022 DP - São Miguel Paulista.xlsx',
 'dp023 DP - Perdizes.xlsx',
 'dp024 DP - Ponte Rasa.xlsx',
 'dp025 DP - Parelheiros.xlsx',
 'dp026 DP - Sacomã.xlsx',
 'dp027 DP - Campo Belo.xlsx',
 'dp028 DP - Freguesia do Ó.xlsx',
 'dp029 DP - Vila Diva.xlsx',
 'dp030 DP - Tatuapé.xlsx',
 'dp031 DP - Vila Carrão.xlsx',
 'dp032 DP - Itaquera.xlsx',
 'dp033 DP - Piritub

In [17]:
#-- creating empty dataframes
db_consolidado = pd.DataFrame()

#-- open and append all xls files
for t in files_xlsx:
    print('C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/' + str(t) )
    data = pd.read_excel('C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/' + t , sep=',', decimal='.')
    db_consolidado = db_consolidado.append(data)

C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp001 DP - Sé.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp002 DP - Bom Retiro.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp003 DP - Campos Elísios.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp004 DP - Consolação.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp005 DP - Aclimação.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp006 DP - Cambuci.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp007 DP - Lapa.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp008 DP - Brás.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp009 DP - Carandiru.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp010 DP - Penha de França.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp011 DP - Santo Amaro.xlsx
C:/Users/tbuzz/OneDrive/Documentos/GitHub/Segurança SP/dp012 DP - Pari.xlsx
C:/Users/tbuzz/OneDrive/Documentos/Gi

In [18]:
#-- printando db_consolidado
db_consolidado.head()

,Unnamed: 0,Natureza,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Total
0,0,HOMICÍDIO DOLOSO (2),2.0,1.0,3.0,1,1.0,2.0,0.0,0.0,1,...,...,...,11.0
1,1,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),2.0,1.0,3.0,1,1.0,2.0,0.0,0.0,1,...,...,...,11.0
2,2,HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.0
3,3,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.0
4,4,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,1.0,0.0,1.0,1,1.0,1.0,0.0,0.0,3,...,...,...,8.0


In [50]:
#-- preenchendo a primeira coluna do db com o nome da delegacia
for i in range(0, db_consolidado.shape[0]):
    t = math.floor(i/23)
    db_consolidado.iloc[i, 0] = db_dps.iloc[t, 0]

In [52]:
#-- printando o db
db_consolidado.tail()

,Unnamed: 0,Natureza,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Total
19,101 DP - Jardim das Imbuias,ROUBO A BANCO,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.000
20,101 DP - Jardim das Imbuias,ROUBO DE CARGA,8.0,11.0,10.0,5,10.0,5.0,8.0,11.0,3,...,...,...,71.000
21,101 DP - Jardim das Imbuias,FURTO - OUTROS,104.0,96.0,230.0,244,196.0,168.0,188.0,189.0,157,...,...,...,1.572
22,101 DP - Jardim das Imbuias,FURTO DE VEÍCULO,21.0,23.0,19.0,24,34.0,30.0,32.0,21.0,25,...,...,...,229.000
0,102 DP - Socorro,HOMICÍDIO DOLOSO (2),0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0,...,...,...,2.000
1,102 DP - Socorro,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0,...,...,...,2.000
2,102 DP - Socorro,HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.000
3,102 DP - Socorro,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.000
4,102 DP - Socorro,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,1.0,0.0,0.0,0,0.0,1.0,0.0,1.0,0,...,...,...,3.000
5,102 DP - Socorro,HOMICÍDIO CULPOSO OUTROS,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,...,...,0.000


In [54]:
#-- alterando o nome da coluna
db_consolidado.rename(columns={'Unnamed: 0': 'Nome DP'}, inplace=True)
db_consolidado.head()

In [59]:
db_dps_filtrado = db_dps[['DPs', 'LatW', 'LongW']]
db_dps_filtrado.head()

,DPs,LatW,LongW
0,001 DP - Sé,-23.5571,-46.6342
1,002 DP - Bom Retiro,-23.5231,-46.6443
2,003 DP - Campos Elísios,-23.5396,-46.6399
3,004 DP - Consolação,-23.5508,-46.6499
4,005 DP - Aclimação,-23.5655,-46.6382


In [67]:
#-- unindo os dbs para trazer a localização geográfica das DPs
db_consolidado_final = pd.merge(left=db_consolidado, right=db_dps_filtrado, left_on='Nome DP', right_on='DPs', how='left')
db_consolidado_final.drop(columns=['DPs'], axis=1, inplace=True)
db_consolidado_final.head()

In [73]:
#-- exportando o database
db_consolidado_final.to_csv('db_consolidado_final_novembro.csv', sep=';', decimal=',', encoding='latin')